In [62]:
%load_ext autoreload
%autoreload 2
from tasrif.processing_pipeline import ProcessingPipeline
from tasrif.processing_pipeline.pandas import ReplaceOperator, DropNAOperator
from tasrif.data_readers.sleep_health import OnboardingDemographicsDataset
import pandas as pd
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
# Full AboutMeDataset
obd = OnboardingDemographicsDataset(shc_folder="../../data/sleephealth/", pipeline=None)
df = obd.raw_df.copy()
print("Shape:", df.shape)

Shape: (8130, 6)


In [57]:
for key in df:
    nas = df[key].isna().sum()
    print("# NA rows for col %s: %d (%.2f%%)" % (key, nas, 100.*nas/df.shape[0]))

# NA rows for col participantId: 0 (0.00%)
# NA rows for col weight_pounds: 0 (0.00%)
# NA rows for col gender: 89 (1.09%)
# NA rows for col height_inches: 0 (0.00%)
# NA rows for col age_years: 54 (0.66%)
# NA rows for col timestamp: 0 (0.00%)


In [59]:
# Analysis to either drop or not censored data
df[df["weight_pounds"] == "CENSORED"] # 431 (5.3%)
df[df["height_inches"] == "CENSORED"] # 337 (4.1%)
df[(df["height_inches"] == "CENSORED") & (df["weight_pounds"] == "CENSORED")] # 254 (3.1%)
df[(df["height_inches"] == "CENSORED") | (df["weight_pounds"] == "CENSORED")] # 514 (6.3%)

,participantId,weight_pounds,gender,height_inches,age_years,timestamp


In [60]:
pipeline = ProcessingPipeline([
    ReplaceOperator(to_replace="CENSORED", value=np.nan),
    DropNAOperator()
    ])

obd = OnboardingDemographicsDataset(shc_folder="../../data/sleephealth/", pipeline=pipeline)
df = obd.processed_dataframe()
df



,participantId,weight_pounds,gender,height_inches,age_years,timestamp
0,3d6c0442-9150-4974-8257-3a94461c8790,190.0,Male,75.0,27.0,2016-07-21T14:44:06-07:00
1,a12e2ca0-2cf4-498f-a51d-1644630511ec,177.0,Female,64.0,19.0,2016-07-21T20:29:50-04:00
2,f63a6a72-88c1-4b49-9326-578f33ed8e9a,190.0,Male,70.0,51.0,2016-03-10T10:34:55-06:00
3,00a347ef-8bac-4b44-8820-338fd910d4e0,150.0,Male,71.0,34.0,2016-03-04T08:15:23-08:00
4,f3f37736-a61f-414e-903a-9de0d92d1ded,136.0,Male,68.0,19.0,2016-04-02T11:50:39-06:00
...,...,...,...,...,...,...
8125,e47e109a-235c-4dd9-b876-f98ca198c967,266.0,Female,64.0,35.0,2019-06-23T21:34:28-04:00
8126,a5fa37f1-de3c-41bd-9324-c31c5244e5c8,270.0,Male,70.0,47.0,2019-06-23T21:35:10-05:00
8127,35610eab-f496-477a-a8d5-d24f3af82726,165.0,Male,72.0,18.0,2019-06-24T23:41:50-04:00
8128,e47e109a-235c-4dd9-b876-f98ca198c967,266.0,Female,64.0,35.0,2019-06-25T16:31:33-04:00
